# Création des 9 DataFrames regroupant les stations par différentes modalités de mesure

Etape 2 du pipeline

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("../data/weatherAUS_preproc.csv")
df.head()

,Unnamed: 0,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,...,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegion,NonMes,CodeRegionNum,NonMesNum
0,0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,...,0,3.141593,3.141593,2.748894,-36.080477,146.91628,NSW,Evaporation_Sunshine,0,0
1,1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,...,0,2.748894,1.963495,3.534292,-36.080477,146.91628,NSW,Evaporation_Sunshine,0,0
2,2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,...,0,3.534292,3.141593,3.534292,-36.080477,146.91628,NSW,Evaporation_Sunshine,0,0
3,3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,...,0,0.785398,5.497787,0.000000,-36.080477,146.91628,NSW,Evaporation_Sunshine,0,0
4,4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,...,0,3.141593,0.392699,2.356194,-36.080477,146.91628,NSW,Evaporation_Sunshine,0,0


In [9]:
################### Code  complet pour créer les 9 tableaux

# Création du dictionnaire clé = station, valeurs = grandeurs non mesurées:
dict = {}
for i in range(df['Location'].unique().shape[0]):  # df['Location'].unique().shape[0]: le nombre de stations
    measures = []
    non_measures = []
    station_df = df.loc[df['Location'] == df['Location'].unique()[i]] #création d'un df pour une station
    for col in station_df.columns:
        if station_df[col].isna().sum()/station_df.shape[0] != 1: # si le pourcentage de nan est différent de 1
            measures.append(col) # on ajoute la grandeur mesurée à 'measures
        else:
            non_measures.append(col) # sinon, on l'ajoute à 'non_measures'
    dict[df['Location'].unique()[i]] = non_measures
    if non_measures == []:
        dict[df['Location'].unique()[i]] = ['all']
    else:
        dict[df['Location'].unique()[i]] = non_measures

# Création de la liste des différentes modalités de non mesure:
modalites = []
for station in dict:
    if dict[station] not in modalites:
        modalites.append(dict[station])

# Création du dictionnaire inverse: clé = liste concaténée des grandeurs non mesurées, valeurs = liste des stations correspondantes
dict_reverse = {}
for i, mod in zip(range(len(modalites)), modalites):
    liste_stations = []
    for station in dict:
        if dict[station] == mod:
            liste_stations.append(station)
    dict_reverse["_".join(modalites[i])] = liste_stations # J'ai appris qu'un dictionnaire ne peut pas prendre de liste en guise de clé. 
                                                          # J'ai décidé de concaténer le nom des colonnes non mesurées avec un "_"       
     

# Création d'une liste contenant les modalités concaténées:
dict_reverse_keys = list(dict_reverse.keys()) 

# Création des 9 fichiers csv:
for key in dict_reverse_keys:
    obj = []
    for i, station in enumerate(dict_reverse[key]):
        stat = df.loc[df['Location'] == station]
        obj.append(stat)
    
    concat = pd.concat(obj, axis = 0) 

    if key != 'all':
        concat_drop = concat.drop(columns = str.split(key, sep = "_"))
    else: concat_drop = concat  # ligne ajoutée pour régler le problème de la création du df 'all'

    title = str(key) + ".csv"
    concat_drop.to_csv(title)
    
# TO DO: 
# 1. choisir des noms de fichiers finaux plus simples que 'Evaporation_Sunshine_Cloud9am_Cloud3pm.csv' ou pas?
# 2. Réaffecter l'addresse de création des fichiers pour les mettre dans '/data'?